In [ ]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# 여의도 경계 설정 및 도로 네트워크 가져오기
ybd_boundary = "Yeouido, Seoul, South Korea"
graph = ox.graph_from_place(ybd_boundary, network_type="walk")  # 보행자 도로 네트워크

# 도로 네트워크에서 노드와 엣지 데이터 추출
nodes, edges = ox.graph_to_gdfs(graph)

# 경사도 추가 (경사도를 계산하는 데 필요한 엣지 데이터 사용)
edges = edges[['u', 'v', 'geometry', 'grade']]  # 경사도만 추출
edges.to_csv("경사도_데이터.csv", index=False)  # 경사도 데이터를 CSV 파일로 저장

# 서울시 보도 통계자료 불러오기
sidewalk_data = pd.read_csv("서울시_보도통계자료.csv")

# 여의도 지역 필터링 (위도, 경도 기준)
ybd_sidewalk = sidewalk_data[
    (sidewalk_data["Y"] >= 37.5195) & (sidewalk_data["Y"] <= 37.5375) &  # 여의도 범위
    (sidewalk_data["X"] >= 126.9225) & (sidewalk_data["X"] <= 126.9395)
]

# 필요한 컬럼 선택 및 저장 (보도폭 정보만 선택)
ybd_sidewalk = ybd_sidewalk[["X", "Y", "보도폭"]]  # 좌표와 보도폭
ybd_sidewalk.to_csv("보도폭_데이터.csv", index=False)  # CSV 파일로 저장

# 공사 현황 데이터 불러오기
construction_data = pd.read_csv("서울시_건설공사_추진현황.csv")

# 여의도 지역 필터링 (위도, 경도 기준)
ybd_construction = construction_data[
    (construction_data["위도"] >= 37.5195) & (construction_data["위도"] <= 37.5375) &
    (construction_data["경도"] >= 126.9225) & (construction_data["경도"] <= 126.9395)
]

# 필요한 컬럼 선택 및 저장 (공사명, 위도, 경도, 상태)
ybd_construction = ybd_construction[["공사명", "위도", "경도", "상태"]]
ybd_construction.to_csv("공사현장_데이터.csv", index=False)  # CSV 파일로 저장

# OpenStreetMap에서 상업 구역 데이터 가져오기
commercial_data = ox.geometries_from_place(ybd_boundary, tags={"landuse": "commercial"})

# 필요한 컬럼 선택 및 저장 (상업 구역)
commercial_data = commercial_data[["geometry"]]  # 상업 구역의 좌표 정보
commercial_data.to_file("상업구역_데이터.geojson", driver="GeoJSON")  # GeoJSON 형식으로 저장

# 데이터 로드 (경사도, 보도폭, 공사 현황 데이터)
slope_data = pd.read_csv("경사도_데이터.csv")  # 경사도 데이터
sidewalk_data = pd.read_csv("보도폭_데이터.csv")  # 보도폭 데이터
construction_data = pd.read_csv("공사현장_데이터.csv")  # 공사 현황 데이터

# 좌표 기준으로 병합 (좌표 간 근사치를 활용한 병합 필요)
# 경사도 데이터와 보도폭, 공사 현황 데이터를 좌표에 맞게 병합
combined_data = slope_data.merge(sidewalk_data, left_on=["u", "v"], right_on=["X", "Y"], how="left")
combined_data = combined_data.merge(construction_data, left_on=["u", "v"], right_on=["경도", "위도"], how="left")

# 상업 구역 데이터 로드 (GeoJSON 형식)
commercial_data = gpd.read_file("상업구역_데이터.geojson")

# 상업 구역의 좌표와 인구 데이터를 병합 (좌표 기준으로 병합)
# 상업 구역을 포함한 인구 밀집도 데이터 결합
combined_data["상업구역"] = combined_data.apply(lambda row: 1 if any(commercial_data.geometry.contains(Point(row["X"], row["Y"]))) else 0, axis=1)

# 위험 점수 계산
# 경사도, 보도폭, 공사 현장 여부, 상업 구역 여부에 따라 위험 점수를 계산
combined_data["위험 점수"] = (
    combined_data["grade"] * 0.124 +  # 경사도 가중치
    combined_data["보도폭"].apply(lambda x: 1/x if x > 0 else 0) * 0.167 +  # 보도폭의 역수 (폭이 좁을수록 위험)
    combined_data["상태"].notna().astype(int) * 0.042 +  # 공사 현장 여부 (상태가 있으면 1)
    combined_data["상업구역"] * 0.148  # 상업 구역 여부 (상업 구역이면 1)
)

# 최종 CSV 파일 저장
combined_data.to_csv("여의도_위험도_분석.csv", index=False)

print("위험도 분석이 완료되어 '여의도_위험도_분석.csv'로 저장되었습니다.")